# Challenge 1:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 2:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [11]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [12]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

nlp = spacy.load('en')

# Parse the cleaned novels. This can take a bit.
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [13]:
def bag_of_words(text):
    
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    return [item[0] for item in Counter(allwords).most_common(1000)]


# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    for i, sentence in enumerate(df['text_sentence']):
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.is_alpha
                     and token.lemma_ in common_words
                 )]
        
        for word in words:
            df.loc[i, word] += 1
            
            if i % 500 == 0:
                print("Processing row {}".format(i))
    
    return df
    
# Set up the bags
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

In [15]:
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)

emma_doc = nlp(emma)

persuasion_sents = [[sent, 'Austen'] for sent in persuasion_doc.sents]
emma_sents = [[sent, 'Austen'] for sent in emma_doc.sents]
emma_sents = emma_sents[0:len(alice_sents)]

emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

NameError: name 'alice_sents' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence', 'text_source'], 1))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
lr = LogisticRegression()
lr.fit(X_train, Y_train)

X_emma_test = np.concatenate((
    X_train[Y_train[Y_train=='Carroll'].index],
    emma_box.drop(['text_sentence', 'text_source'], 1)
), axis=0)

Y_emma_test = pd.concat([
    Y_train[Y_train=='Carroll'],
    pd.series(['Austen'] * emma_bow.shape[0])])

print('Test set score:', lr.score(X_emma_test, Y_emma_test))
pd.crosstab(Y_emma_test, lr.predict(X_emma_test))

In [ ]:
from sklearn import svm
svm = svm.SVC()
svm.fit(X_train, Y_train)
print('SVM accuracy score:', svm.score(X_emma_test, Y_emma_test))
pd.crosstab(Y_emma_test, svm.predict(X_emma_test))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm, X_emma_test, Y_emma_test, cv=10)

In [ ]:
pnp = gutenberg.raw('pride-and-prejeduce.txt')